<html> <img style="float:center; margin: 0px 0px 0px 0px;" src="https://docs.google.com/uc? export=download&id=1jIxUCriI-tyCAkUYHz05rk5SY_rTHRV9" width="220" align="left"/> </html> 

<figure>
<center>
<img src='https://docs.google.com/uc?export=download&id=1BmLEthy7KCNsnVFviFKX77zLmFhc47XZ' align='right'width="220"/>
    
<figcaption>

</figcaption></center>
</figure>
<center> <H1> Analyse et Programmation avec Python </H1> </center>



Enseignant : Jaafar Chaaouri

Email: Jaafar.chaaouri@fsm.rnu.tn

---

# **Analyse des ventes**

## Introduction

Dans le cadre de ce projet, nous souhaitons réaliser une analyse des données d'une plateforme de e-commerce fictive afin de déterminer de nouvelles stratégies marketing. Pour ce faire, il faut d'abord restructuré et nettoyé les données issues du SI et les analysées selon trois axes.

- Etudier l'évolution du chiffre d'affaires au cours du temps afin de déterminer des effets saisonniers et d'éventuels produits régulateurs.

- Analyser la distribution des catégories de produits et la proportion des ventes de chaque produit (courbe de Lorenz, loi des 80-20).

- Enfin, examiner les corrélations entre l'âge, le sexe, les dépenses, les fréquences d'achats et le panier des clients. Cela nous permettra d'établir trois profils types de clients détaillés et de suggérer des stratégies de ventes.






Organisation des sources


- dataset : les données initiales du projet
- clean_dataset : les données nettoyées du projet

graphiques : l'ensemble des graphiques du projet. 



- preparation_de_donnees.ipynb : récupère les données de dataset, les nettoie, et enregistre le résultat dans  clean_dataset
- analyse_de_donnees.ipynb : analyse les données qu'il y a dans clean_dataset






---



In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import re
#import functions as f
import calendar
import math

# Mission 1 : Préparation des données

# Découverte des données

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [7]:
c = pd.read_csv("/content/drive/My Drive/customers.csv")
p = pd.read_csv("/content/drive/My Drive/products.csv")
t = pd.read_csv("/content/drive/My Drive/transactions.csv")

In [8]:
c.shape

(8623, 3)

In [9]:
c.dtypes

client_id    object
sex          object
birth         int64
dtype: object

In [10]:
c.head(2)

,client_id,sex,birth
0,c_4410,f,1967
1,c_7839,f,1975


In [ ]:
c.info()

### `customers`

Contient **~8600** clients

colonnes :
- `client_id` est une clef primaire
- `sex` : **f** ou **m** (52% et 48%)
- `birth`: 1929 -> 2004

Autres :
- pas de valeurs nulles

In [ ]:
c.dtypes

In [ ]:
c['sex'].value_counts()

In [ ]:
c.sex[0:3]
# ou c['sex'].head()

### `products`

In [ ]:
# p.shape 
p.describe()

In [ ]:
p.count()

In [ ]:
p.info()

In [ ]:
p.describe()

In [ ]:
p.head(2)

Contient **~3200** produits

colonnes :
- `id_prod`: clef primaire
- `categ`: 0, 1 ou 2 (70%, 23%, 7%)

Autres :
- pas de valeurs nulles

In [ ]:
# p['categ'].describe() 
p['categ'].value_counts() 

### `Transactions`

il y a **~340 000** transactions

In [ ]:
t.head(2)

Colonnes :
- `id_prod` : id du produit acheté
- `client_id` : id de l'acheteur
- `session_id` : la session pendant laquelle laquelle le client a acheté
- `date` : la date d'achat

Autre :
 - il n'y a pas de valeurs nulles
 - Il y a un utilisateur par sessions (sauf pour s_0 qui a ct_0 et ct_1). Mais un utilisateur peut avoir plusieurs sessions
 - ct_0 et ct_1 posent problème.
    - Ils ont acheté T_0 126 fois à partir de la meme session (s_0) a ~2021-03-01 02:30:02.2374.

In [ ]:
t.describe()

In [ ]:
# Accès indicé aux données d'un DataFrame 
t.iloc[::10, ]
# t.shape

In [ ]:
t.loc[t['session_id']=='s_0']
#t.loc[1431]

# Jointure

On fait une  jointure externe pour sauvegarder l'ensembles des produits / clients qui n'ont pas de transactions

In [11]:
# jointure des transac vers les produits
df = pd.merge(left=p,right=t,how='outer',on='id_prod')


In [13]:
df.head()

,id_prod,price,categ,date,session_id,client_id
0,0_1421,19.99,0.0,2021-05-12 23:26:29.885083,s_33709,c_7954
1,0_1421,19.99,0.0,2022-01-25 11:27:39.357012,s_154957,c_254
2,0_1421,19.99,0.0,2022-01-02 22:29:33.796003,s_144068,c_5770
3,0_1421,19.99,0.0,2021-03-05 17:10:40.265676,s_2187,c_1455
4,0_1421,19.99,0.0,2021-03-29 13:55:04.422637,s_13112,c_2768


In [ ]:
# jointures des transac/prod vers c
df = pd.merge(left=c,right=df, how='outer',on='client_id')

In [ ]:
# renommage des colonnes
df.columns = ['c_id', 'c_sex', 'c_birth', 'p_id', 'p_price', 'p_cat', 't_date', 't_sess_id']

In [ ]:
df.head()

In [ ]:
df.c_id.isin(['ct_0', 'ct_1'])

In [ ]:
df[df.c_id.isin(['ct_0', 'ct_1'])]

In [ ]:
df[df.c_id.isin(['ct_0', 'ct_1'])].sample(3)

In [ ]:
df.shape

In [ ]:
df['c_id'].value_counts()

#### Y a-t-il des valeurs `NaN` ?

In [ ]:
df_na = df.loc[df.isnull().any(axis=1)]

In [ ]:
df_na

In [ ]:
df_na.info()

In [ ]:
df.c_id.isnull()

In [ ]:
p_invendus_id = df_na.loc[df.c_id.isnull(), 'p_id'].unique()

In [ ]:
p_invendus_id

In [ ]:
p_invendus_id.shape

In [ ]:
c_ss_achat_id = df_na.loc[df.p_id.isnull(), 'c_id'].unique()
c_ss_achat_id.shape 

- 22 produits sur 3289 n'ont jamais été vendus
- 21 clients sur 8624 n'ont jamais acheté

# Nettoyage des données

#### On ajoute une colonne 'data_info'

elle donne des infos supplémentaire sur les données de la ligne (`NaN` par défaut) 

In [ ]:
df['data_info'] = np.nan

#### Suppression des valeurs de tests

Les valeurs de tests :
 - les clients sont ct_0 ou ct_1
 - la session de la transaction est s_0
 - les dates de la trans comences par 'test'
 - les produits acheté T_0, coutent -1 euro
 
 --> on les supprime

In [ ]:
# df2 = df.loc[(df.c_id.notnull() & (df.c_id.str.contains('ct')))]
df = df.loc[~(df.c_id.notnull() & (df.c_id.str.contains('ct')))]

In [ ]:
df

#### Le produit `0_2245` est dans des transactions mais n'est pas dans produits

le prermier caractère de l'id correspond t'il a la catégorie ?

In [ ]:
test = df[['p_id', 'p_cat']].copy()
test['equal'] = test['p_id'].str[0] == test['p_cat'].astype(str).str[0]

In [ ]:
test[test['p_id'].notnull()& test['p_cat'].notnull()].equal.all()

--> oui on peut donc faire une imputation de la catégorie a partir de l'id

In [ ]:
# imputation de la catégorie
cat = 0
df.loc[df.p_id=='0_2245', 'p_cat'] = cat

Et l'on peut imputer le prix par la moyenne de la catégorie

In [ ]:
cat1_price_mean = df.loc[df.p_cat == cat, 'p_price'].mean()
df.loc[df.p_id=='0_2245', 'p_price'] = cat1_price_mean

In [ ]:
df.loc[df.p_id=='0_2245', 'data_info'] = 'prod_imputation'

In [ ]:
df.loc[df.p_id=='0_2245', ['p_id', 'p_cat', 'p_price']].drop_duplicates()

#### On transforme les date en objets *Date*

In [ ]:
def toDate(val):
    if pd.isnull(val):
        return val
    else:
        return datetime.strptime(val, '%Y-%m-%d %H:%M:%S.%f')

In [ ]:
df['t_date'] = df['t_date'].apply(toDate)

In [ ]:
df.head(1)

#### On transforme en string les cat de produits

In [ ]:
df['p_cat'] = df['p_cat'].fillna(-1)
df['p_cat'] = df['p_cat'].apply(lambda x: str(int(x)))
df['p_cat'] = df['p_cat'].replace('-1', np.nan)

#### On détermine l'age des acheteurs

In [ ]:
df['t_date'].dt.year

In [ ]:
df['t_buyer_age'] = df['t_date'].dt.year - df['c_birth']

#### on discrétise l'age des acheteurs

In [ ]:
def discretise_age(age):
    tranches = [
        {'bornes': (0,30), 'label': '30-'},
        {'bornes': (31,50), 'label': '31-50'},
        {'bornes': (51,70), 'label': '51-70'},
        {'bornes': (71,120),'label': '71+'}
    ]
    for t in tranches:
        if t['bornes'][0] <= age <= t['bornes'][1]:
            return t['label']

In [ ]:
df['t_buyer_age_range'] = df['t_buyer_age'].apply(discretise_age)

#### On calcule l'age des consomateurs

In [ ]:
annee = 2022
df['c_age']  = annee - df['c_birth']

#### on discrétise l'age des consommateurs

In [ ]:
df['c_age_range'] = df['c_age'].apply(discretise_age)

#### L'âge de 18 an est surreprésenté

In [ ]:
df[['c_id', 'c_age']].drop_duplicates().c_age.plot('hist', bins=100)
plt.plot()

on ajoute un flag 'age irrelevant'

In [ ]:
df.loc[df.c_age == 18, 'data_info'] = 'age_irrelevant'

#### On exprime la date de différentes manières

In [ ]:
# mois d'achat
df['t_month'] = df['t_date'].dt.month
# début, fin du mois ? 0,1,2,3
df['t_month_part'] = (df['t_date'].dt.day / (df['t_date'].dt.days_in_month/4+1)).apply(lambda x: str(x)[0])
# jour de la semaine
df['t_weekday'] = df['t_date'].dt.dayofweek
# heure d'achat
df['t_hour'] = df['t_date'].dt.hour

#### On discrétise le prix

In [ ]:
def discretise_price(price):
    tranches = [
        {'bornes': (0,4),  'label': 'very_cheap 3-'},
        {'bornes': (4,10), 'label': 'cheap 4-9'},
        {'bornes': (10,20), 'label': 'medium 10-19'},
        {'bornes': (20,80), 'label': 'expensive 20-79'},
        {'bornes': (80,500), 'label': 'very_expansive 80+'}
    ]
    for t in tranches:
        if t['bornes'][0] <= price < t['bornes'][1]:
            return t['label']

In [ ]:
df['p_price_range'] = df['p_price'].apply(discretise_price)

<hr>

#### Enfin, renommage des colonnes

In [ ]:
df = df[[
    'c_id',
    'c_sex',
    'c_birth',
    'c_age',
    'c_age_range',
    
    'p_id',
    'p_cat',
    'p_price',
    'p_price_range',
    
    't_sess_id',
    't_buyer_age',
    't_buyer_age_range',
    
    't_date',
    't_hour',
    't_weekday',
    't_month_part',
    't_month',
    
    'data_info'
]]

# Projection et enregistrement

on créé 3 dataframe : 
- les produits invendus
- les cliens qui n'ont rien acheté
- toutes les transactions avec information des clients, 

In [ ]:
produits_invendus = df.loc[df['p_id'].isin(p_invendus_id), ['p_id', 'p_cat', 'p_price', 'p_price_range']]

In [ ]:
clients_sans_achat = df.loc[df['c_id'].isin(c_ss_achat_id), ['c_id', 'c_sex', 'c_birth', 'c_age', 'c_age_range']]

In [ ]:
transactions = df.dropna(subset=['c_id', 'c_sex', 'c_birth', 'c_age', 'c_age_range', 'p_id', 'p_cat',
       'p_price', 'p_price_range', 't_sess_id', 't_buyer_age',
       't_buyer_age_range', 't_date', 't_hour', 't_weekday', 't_month_part',
       't_month'])

#### Enregistrement des données

In [ ]:
produits_invendus.to_csv('/content/drive/My Drive/dataset/clean_dataset/produits_invendus.csv', index=False)
clients_sans_achat.to_csv('/content/drive/My Drive/dataset/clean_dataset/clients_sans_achat.csv', index=False)
transactions.to_csv('/content/drive/My Drive/dataset/clean_dataset/transactions.csv', index=False)